In [1]:
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import time
from sklearn import metrics
path='dataset//reddit_train.csv'
path_test='dataset//reddit_test.csv'
labels=['Overwatch','Music','conspiracy','AskReddit','worldnews','anime','hockey','baseball','funny','nba',
    'canada','soccer','nfl','trees','gameofthrones','wow','leagueoflegends','GlobalOffensive','europe','movies']


In [2]:
data = pd.read_csv(path)
data.head(5)

,id,comments,subreddits
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey
1,1,Ah yes way could have been :( remember when he...,nba
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends
3,3,He wouldn't have been a bad signing if we woul...,soccer
4,4,Easy. You use the piss and dry technique. Let ...,funny


In [3]:
print(data.columns)                         
print(data.shape)                           
print(data.loc[0:2])                        
categories=['']
data['subreddits'].value_counts()

Index(['id', 'comments', 'subreddits'], dtype='object')
(70000, 3)
   id                                           comments       subreddits
0   0  Honestly, Buffalo is the correct answer. I rem...           hockey
1   1  Ah yes way could have been :( remember when he...              nba
2   2  https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...  leagueoflegends


AskReddit          3500
Music              3500
GlobalOffensive    3500
worldnews          3500
conspiracy         3500
soccer             3500
wow                3500
movies             3500
trees              3500
Overwatch          3500
leagueoflegends    3500
nba                3500
nfl                3500
baseball           3500
gameofthrones      3500
anime              3500
europe             3500
funny              3500
canada             3500
hockey             3500
Name: subreddits, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data['comments'], data['subreddits'], train_size=0.8, test_size=0.2)
print(len(X_train))

56000


In [6]:
vectorizer = CountVectorizer(stop_words='english')
vectors_train = vectorizer.fit_transform(X_train)

vectors_test = vectorizer.transform(X_test)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer(sublinear_tf = True)
vectors_train_idf = tf_idf_vectorizer.fit_transform(X_train)
vectors_test_idf = tf_idf_vectorizer.transform(X_test)

In [8]:
class BNB():
    
    def __int__(self, labels):
        self.labels=labels
        pass
    
    def prior(self, vectors_train, y_train):        
        ##y_prior
        y_prior=np.zeros(len(labels))
        y_num=np.zeros(len(labels))
        for i in range(len(y_prior)):
            y_num[i]=y_train.value_counts()[labels[i]]
        y_prior=y_num/sum(y_num)
        
        ##x_prior
        y_train_m=y_train.as_matrix()## series to matrix
        lens=vectors_train.shape[1]
        features=np.zeros((len(labels),lens))
        for (index,data) in enumerate(vectors_train):
            a=labels.index(y_train_m[index])
            features[a, data.indices]+=1
        return y_prior, features, y_num
    
    def cond_prob(self, vectors_train, y_train):
        y_prior, features, y_num=self.prior(vectors_train, y_train)
        y_num_sm=y_num+2  #smoothing
        features_sm=features+1
        lens=vectors_train.shape[1]
        
        theta_1=np.zeros((len(labels),lens))
        theta_0=np.zeros((len(labels),lens))

        x_sum=np.zeros(lens)#total # of xj
        y_sum=sum(y_num)#total # of samples

        for i in range(lens):
            x_sum[i]=sum(features[:,i])

        for i in range(len(labels)):
            theta_1[i]=features_sm[i]/y_num_sm[i]
            theta_0[i]=(x_sum-features[i]+1)/(y_sum-y_num[i]+2)
            
        return theta_1, theta_0, y_prior
    
    def fit(self, vectors_train, y_train):
        lens=vectors_train.shape[1]
        theta_1, theta_0, y_prior=self.cond_prob(vectors_train, y_train)
        wj_1=np.zeros((len(labels),lens))
        wj_0=np.zeros((len(labels),lens))
        wj_1=np.log(theta_1)-np.log(theta_0)
        wj_0=np.log(1-theta_1)-np.log(1-theta_0)
        
        w0=np.zeros(len(labels))
        w0=np.log(y_prior)-np.log(1-y_prior)+wj_0.sum(axis=1)
        return w0, wj_1, wj_0
    
    def predict(self, vectors_train, y_train, vectors_test):
        w0, wj_1, wj_0=self.fit(vectors_train, y_train)
        m,n=vectors_test.shape
        feature_log=np.zeros(len(labels))
        y_pre=np.zeros(m)
        xx_binary=np.zeros(n)
        for (index,data) in enumerate(vectors_test):
            xx_binary[:]=0 ##binarize features
            xx_binary[data.indices]=1
            X_w=np.dot(wj_1-wj_0,xx_binary)
            feature_log=w0+X_w
            y_pre[index]=np.argmax(feature_log)

        return y_pre
    

In [9]:
time_start=time.time()
clf = BNB()
y_pre=clf.predict(vectors_train, y_train, vectors_test)
time_end=time.time()
print('time cost(s): ',time_end-time_start)
labels_array=np.array(labels)
accuracy=(labels_array[y_pre.astype(np.int)]==y_test).astype(np.int).mean()
print("test accuracy: %0.2f%%" %(accuracy*100))

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


time cost(s):  110.97039556503296
test accuracy: 53.28%


In [10]:
from sklearn.naive_bayes import BernoulliNB
time_start=time.time()
clf_B = BernoulliNB()
clf_B.fit(vectors_train, y_train)
y_pred_B=clf_B.predict(vectors_test)
time_end=time.time()
print("test accuracy: %0.2f%%" %((y_pred_B==y_test).astype(np.int).mean()*100))
print('time cost(s): ',time_end-time_start)

test accuracy: 49.71%
time cost(s):  0.39664673805236816
